This notebook shows how to create a Threshold Crossing Event (TCE) object using the framework built into exovetter. 

A TCE stores the measured properties (orbital period, transit depth, etc.) of a proposed transit that usually come out of a transit search pipeline. You can create TCEs by manually inputting the data, or automattically populating the required fields using exo.mast's API. TCEs are used as an input to most vetters.


In [3]:
%load_ext autoreload
%autoreload 2
from astropy import units as u
from exovetter import const as exo_const
from exovetter.tce import Tce

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Example: Creating a TCE in TESS BKJD by manually inputing properties:

The most straightforward way to create a TCE is by manually populating the fields with values.

In [4]:
# Define properties which will be used in the TCE
period = 5.3 * u.day
epoch = 133.4 * u.day
depth = 1 * exo_const.ppm
duration = 24 * u.hr

# create a TCE object using the exovetter.tce.Tce class
my_tce = Tce(period=period, epoch=epoch, 
             epoch_offset=exo_const.bkjd, depth=depth, 
             duration=duration, comment='example')

display(my_tce)
    

{'period': <Quantity 5.3 d>,
 'epoch': <Quantity 133.4 d>,
 'epoch_offset': <Quantity -2454833. d>,
 'depth': <Quantity 1.e-06>,
 'duration': <Quantity 24. h>,
 'comment': 'example'}

You can check if all required parameters are present to run in a vetter using the validate method

In [5]:
if my_tce.validate() is True:
    print('All required parameters are present')
else:
    print('All required parameters are not present')

All required parameters are present


## Example: Creating a TCE from archival data

For objects in the exo.MAST archive (https://exo.mast.stsci.edu/docs/) we can pull down data to populate the TCE object.

In [20]:
import requests

planeturl = "https://exo.mast.stsci.edu/api/v0.1/exoplanets/"
planet_name = "WASP-18 b"
url = planeturl + planet_name + "/properties/"

header={}
r = requests.get(url = url, headers = header)

planet_props = r.json() # return a list of properties from the catalogs found on mast 

print('catalogs found:')
for catalog in planet_props:
    print(catalog['catalog_name'])
    

catalogs found:
exoplanets.org
nexsci


These values taken from the archive can then be input into the TCE object.

In [25]:
nexsci_prop = planet_props[1] 

my_tce = Tce(period = nexsci_prop['orbital_period'] * u.day,
             epoch = nexsci_prop['transit_time'] * u.day,
             epoch_offset =  -2_400_000.5 * u.day,
             depth = nexsci_prop['transit_depth'] * exo_const.frac_amp,
             duration = nexsci_prop['transit_duration'] * u.day,
             snr = 50, #This value is only used by LPP and doesn't need to be very precise
             event_name = "WASP-18 b",
             target_name = "WASP-18")

display(my_tce)

{'period': <Quantity 0.94145223 d>,
 'epoch': <Quantity 58374.669883 d>,
 'epoch_offset': <Quantity -2400000.5 d>,
 'depth': <Quantity 0.00931894>,
 'duration': <Quantity 0.08932 d>,
 'snr': 50,
 'event_name': 'WASP-18 b',
 'target_name': 'WASP-18'}

# Example: Automatically creating a TCE from archival data

This process can be automated using the utils function `get_mast_tce()`. This populates the TCE with the period, epoch, epoch_offset, depth, and duration from exo.MAST. If multiple catalogs are present it will provide a list of TCEs created from each catalog.

In [31]:
from exovetter import utils

candidate = "WASP-18 b"
tce_list = utils.get_mast_tce(candidate)

display(tce_list)

[{'period': <Quantity 0.9414529 d>,
  'epoch': <Quantity 54664.40531 d>,
  'epoch_offset': <Quantity -2400000.5 d>,
  'depth': <Quantity 0.00916>,
  'duration': <Quantity 0.09089 d>,
  'target': 'WASP-18 b',
  'catalog_name': 'exoplanets.org'},
 {'period': <Quantity 0.94145223 d>,
  'epoch': <Quantity 58374.669883 d>,
  'epoch_offset': <Quantity -2400000.5 d>,
  'depth': <Quantity 0.00931894>,
  'duration': <Quantity 0.08932 d>,
  'target': 'WASP-18 b',
  'catalog_name': 'nexsci'}]

## Example: Saving and opening a saved TCE

TCE objects have a `to_json()` method which can be used to save a created TCE to a json file.

In [32]:
my_tce.to_json('wasp18b_tce')

'{"period": 0.94145223, "period_unit": "d", "epoch": 58374.6698830002, "epoch_unit": "d", "epoch_offset": -2400000.5, "epoch_offset_unit": "d", "depth": 0.009318937404608593, "depth_unit": "", "duration": 0.08932, "duration_unit": "d", "snr": 50, "event_name": "WASP-18 b", "target_name": "WASP-18"}'

These can then be read into a new TCE using the `from_json()` method

In [34]:
my_tce = Tce.from_json('wasp18b_tce')
display(my_tce)

{'period': <Quantity 0.94145223 d>,
 'epoch': <Quantity 58374.669883 d>,
 'epoch_offset': <Quantity -2400000.5 d>,
 'depth': <Quantity 0.00931894>,
 'duration': <Quantity 0.08932 d>,
 'snr': 50,
 'event_name': 'WASP-18 b',
 'target_name': 'WASP-18'}